In [30]:
#import bokeh.sampledata

In [21]:
from basic_tools import *
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import json

In [22]:
from bokeh.io import output_file, show
from bokeh.embed import json_item
from bokeh.plotting import figure

from bokeh.transform import cumsum
from bokeh.models import Plot, Range1d, Circle, MultiLine
from bokeh.models import OpenURL
from bokeh.models import TapTool, PanTool, WheelZoomTool,WheelPanTool, HoverTool, BoxZoomTool, ResetTool,LassoSelectTool, UndoTool, RedoTool,SaveTool,ZoomOutTool
from bokeh.embed import components

# Loading variables and basic functions

In [4]:
h2_total=pd.read_csv(h2_total_par_filtered_path,index_col='phenotype')
h2_total.shape

(497, 7622)

In [5]:
suffix_bp_list_power="bp300 bp128 bp64 bp32 bp16 bp8".split()

In [6]:
regression_result_bp_list_dict=dict([make_regression_result_list(h2_total,phenotype,suffix_bp_list_power,False) for phenotype in phenotypes_par_filtered.index])

In [7]:
def calculate_per_bp_var(regression_result_list_dict,suffix_list,mode='default'):
    var_list=[]
    for idx,value in enumerate(suffix_list):
        h2_df=pd.DataFrame([[phenotype]+((regression_result_list[idx]['h2']/regression_result_list[idx]['h2'].sum())/regression_result_list[idx]['category'].apply(category_to_format,args=('bp_range',))).tolist() for phenotype,regression_result_list in regression_result_list_dict.items()])
        #h2_df=pd.DataFrame([[phenotype]+((regression_result_list[idx]['h2']/regression_result_list[idx]['h2'].sum())/1).tolist() for phenotype,regression_result_list in regression_result_list_dict.items()])
        h2_df=h2_df.set_index(0)

        if mode=='default':
            values=h2_df.var(axis=1)
        elif mode=='mono':
            h2_mono=pd.Series(0.0,index=np.arange(h2_df.shape[1]));h2_mono[0]=1/category_to_format(list(regression_result_bp_list_dict.values())[0][idx]['category'].loc[0],'bp_range',)
            values=h2_df.var(axis=1)/h2_mono.var()
        elif mode=='max_and_poly':
            h2_mono=pd.Series(0.0,index=np.arange(h2_df.shape[1]));h2_mono[0]=1/category_to_format(list(regression_result_bp_list_dict.values())[0][idx]['category'].loc[0],'bp_range',)
            h2_poly=pd.Series(1.0,index=np.arange(h2_df.shape[1]));h2_poly=h2_poly/list(regression_result_bp_list_dict.values())[0][idx]['category'].apply(category_to_format,args=('bp_range',)).sum()
            #var_list.append(1-(h2_df.var(axis=1)-h2_poly.var())/(h2_df.var(axis=1).max()-h2_poly.var()))#-h2_df.mean(axis=1))
            values=(h2_df.var(axis=1)-0)/(h2_df.var(axis=1).max()-0)
        elif mode=='test':
            values=-np.log10(h2_df.var(axis=1))
            8.580318	
            values=(values.max()-values)
            values=1-values/values.max()
            #values=values-values.min()
            #values=1-(values-values.min())/(values.max()-values.min())
        else:
            raise
        var_list.append(values)
    var_df=pd.DataFrame(var_list,index=suffix_list)
    return var_df.T
per_bp_var=calculate_per_bp_var(regression_result_bp_list_dict,suffix_bp_list_power,mode='test')
per_bp_var.sort_values('bp8',ascending=False).head()

found that variables 'gwas_result_sample','chr_bp_max' were not loaded. trying to load the variables
gwas_result_sample.shape: (13364303, 18)


,bp300,bp128,bp64,bp32,bp16,bp8
0,,,,,,
6138_1,1.000000,0.898765,1.000000,1.000000,0.994746,1.000000
23115_irnt,0.871682,0.754938,0.862953,0.900980,0.980785,0.989842
23119_irnt,0.832710,0.777364,0.862892,0.898101,0.983032,0.982370
23104_irnt,0.825578,0.799314,0.894048,0.903790,0.991330,0.979252
2139_irnt,0.935974,0.763283,0.866459,0.888004,0.979229,0.978693


In [8]:
def regression_result_groupchr(regression_result):
    regression_result_temp=regression_result.copy()
    regression_result_temp['h2_ratio']=regression_result_temp['h2']/regression_result_temp['h2'].sum()
    regression_result_temp['chr']=regression_result_temp['category'].apply(lambda x:category_to_format(x,'chr'))
    regression_result_temp_chr=dict([(idx,regression_result_temp.loc[value]) for idx,value in regression_result_temp.groupby('chr').groups.items()])    
    return regression_result_temp_chr

In [9]:
def absorb_negative(array):
    array=np.array(array)
    for i in range(len(array)):
        if array[i]<0:
            if i<len(array)-1:
                array[i+1]=array[i+1]+array[i]
                array[i]=0
            else:
                array[i]=0
    return array

# For bokeh

In [48]:
def heatmap_to_bokehdata(df):
    if df.columns.duplicated().sum()>0:
        df=df.iloc[:,~df.columns.duplicated()]
        print("duplicated column deleted")
    if df.index.duplicated().sum()>0:
        df=df.iloc[~df.index.duplicated(),:]
        print("duplicated index deleted")
    
    df_bokeh=pd.DataFrame(np.array([[(code,idx,round(value,3) if type(value)==float else value) for code,value in row.iteritems()] for idx,row in df.iterrows()]).reshape(-1,3),columns=['x_data','y_data','value'])
    if np.isreal(df.iloc[1,1]):
        df_bokeh['value']=df_bokeh['value'].astype(float)

    return df_bokeh

def norm_color(array,cmap,tohex=True,tonorm=True):
    array=np.array(array)
    #print(array)
    if tonorm:
        array=(array-array.mean())/array.std()
    array=[cmap(value) for value in array]
    #array=np.vectorize(lambda x: plt.cm.viridis(x))(array)    
    if tohex:
        array=["#%02x%02x%02x"%(int(256*color[0]),int(256*color[1]),int(256*color[2])) for color in array]
    return array

#heatmap_to_bokehdata(h2_bp8_df.T.corr())
#anova_bokeh['color']=(anova_bokeh['value']-anova_bokeh['value'].mean())/anova_bokeh['value'].std()

#anova_bokeh['color']=anova_bokeh['color'].map(lambda x: np.array(plt.cm.viridis(x)))
#anova_bokeh['color']=anova_bokeh['color'].map(lambda color: "#%02x%02x%02x"%(int(256*color[1]),int(256*color[0]),int(256*color[2])))

In [11]:
def bokeh_to_html(p,path):
    script, div = components(p)
    with open(path+'_div','w') as f:
        f.write(div)
    with open(path+'_script','w') as f:
        f.write(script)  

## Template
```
from bokeh.io import output_file, show
from bokeh.embed import json_item
from bokeh.plotting import figure

from bokeh.transform import cumsum
from bokeh.models import Plot, Range1d, Circle, MultiLine
from bokeh.models import TapTool, PanTool, WheelZoomTool,WheelPanTool, HoverTool, BoxZoomTool, ResetTool,LassoSelectTool, UndoTool, RedoTool,SaveTool,ZoomOutTool

p = figure(plot_height=700, title="Landscape of Heritability",
    x_range=(-1.0, 1.0),y_range=(-1.0, 1.0),tools="")

p.~~~

hover_tool = HoverTool(tooltips="@category <br> @value2 (@value1 %)")
p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())

p.title.text = "Correlation Network"

p.axis.visible=False
p.xaxis.major_label_orientation = "vertical"
#p.xaxis.major_label_orientation='vertical'

p.xgrid.visible = False
p.xgrid.grid_line_color = None
p.ygrid.visible = False

p.legend.visible=False

#p.y_range.start = 0
p.y_range.start = 0
fig.y_range.flipped = True
```

# Configuration

In [12]:
web_path='web/partitioned_heritability_website/'

web_data_path='web/partitioned_heritability_website/plot_data/'
web_test_path=web_data_path+'test.html'

web_network_path=web_data_path+'network'
web_poly_h2_all_path=web_data_path+'poly_h2_all'
web_poly_h2_cont_path=web_data_path+'poly_h2_cont'
web_poly_h2_bin_path=web_data_path+'poly_h2_bin'
web_anova_total_path=web_data_path+'anova_total'
web_byregion_path=web_data_path+'byregion'

web_phenotype_corr_rank_path=web_data_path+'phenotype_corr_rank_{}'
web_phenotype_anova_phenotype_path=web_data_path+'phenotype_anova_phenoype_{}'
#web_phenotype_anova_phenotype_path=web_path+'anova_phenoype_{}.json'
web_phenotype_par_pie_path=web_data_path+'phenotype_par_pie_{}'

# Network

In [114]:
h2_bp8_df=pd.DataFrame([[phenotype]+(regression_result_list[-1]['h2']/regression_result_list[-1]['h2'].sum()).tolist() for phenotype,regression_result_list in regression_result_bp_list_dict.items()])
h2_bp8_df=h2_bp8_df.set_index(0)

In [115]:
h2_bp8_df_temp=h2_bp8_df.copy()
edges=h2_bp8_df_temp.T.corr().mask(np.tril(np.ones(h2_bp8_df_temp.T.corr().shape)).astype(np.bool)).stack()
edges=pd.DataFrame([[edge[0][0],edge[0][1],edge[1]] for edge in edges.iteritems()],columns=['source','target','value'])
pheno_unique=h2_bp8_df.index.difference(pd.Index(edges[edges['value']>0.95]['target'].unique()))

In [116]:
len(pheno_unique),h2_bp8_df.shape

(481, (497, 358))

In [117]:
h2_bp8_df_temp=h2_bp8_df.copy().loc[pheno_unique]
h2_bp8_df_temp=h2_bp8_df_temp.loc[~h2_bp8_df_temp.index.duplicated()]

In [118]:
edges=h2_bp8_df_temp.T.corr().mask(np.tril(np.ones(h2_bp8_df_temp.T.corr().shape)).astype(np.bool)).stack()
edges=pd.DataFrame([[edge[0][0],edge[0][1],edge[1]] for edge in edges.iteritems()],columns=['source','target','value'])

In [119]:
edges.shape

(115440, 3)

In [120]:
#edges_filtered=edges.loc[(edges['value'] <0.8) & (edges['value'] >0.5)]# & (edges['source'] != edges['target'])]\
edges_filtered=edges.loc[(edges['value'] >0.5)]# & (edges['source'] != edges['target'])]

In [121]:
groups=[]
for idx,row in edges_filtered.iterrows():
    try:
        group_index=[(row['source'] in groups[i] or row['target'] in groups[i]) for i in range(len(groups))].index(True)
        groups[group_index].add(row['source'])
        groups[group_index].add(row['target'])
    except:
        groups.append({row['source'],row['target']})

In [122]:
groups=sorted(groups,key=lambda x: len(x),reverse=True)

In [123]:
edges_filtered['group_id']=[np.argmax([False]+[row['source'] in group for group in groups]) for idx,row in edges_filtered.iterrows()]

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


edges_filtered.source=edges_filtered.source.map(lambda x:description_to_short(code_to_description(x),'abbr_pub_ori'))
edges_filtered.target=edges_filtered.target.map(lambda x:description_to_short(code_to_description(x),'abbr_pub_ori'))

In [139]:
import networkx as nx

In [140]:
edges_filtered_filtered=edges_filtered[edges_filtered['group_id']==1]
G=nx.convert_matrix.from_pandas_edgelist(edges_filtered_filtered,edge_attr='value')

In [141]:
per_bp_var_temp=per_bp_var.copy()
nx.set_node_attributes(G,dict([(node,pd.Series(per_bp_var_temp['bp8'].loc[node]).iloc[0]) for node in list(G.nodes)]),'per_bp_var')

In [142]:
nx.set_node_attributes(G,dict([(node,pd.Series(node).iloc[0]) for node in list(G.nodes)]),'phenotype')

In [143]:
nx.set_node_attributes(G,dict([(node,pd.Series(node).iloc[0].lower()) for node in list(G.nodes)]),'phenotype_lower')

In [145]:
#G=nx.relabel_nodes(G,lambda x:description_to_short(code_to_description(x),'abbr_pub_ori',2))
G=nx.relabel_nodes(G,lambda x:description_to_short(code_to_description(x),'abbr_pub_ori','correlation'))
pos = nx.kamada_kawai_layout(G,weight='value')

In [149]:
G.nodes['Time watching TV']

{'per_bp_var': 0.9519654578938342,
 'phenotype': '1070',
 'phenotype_lower': '1070',
 'per_bp_var_round': '0.952'}

In [150]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, Circle, HoverTool, MultiLine, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx,NodesAndLinkedEdges, EdgesAndLinkedNodes
#from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool

from bokeh.plotting import figure

for node in G.nodes:
    G.nodes[node]['per_bp_var_round']='{:.3f}'.format(G.nodes[node]['per_bp_var'])

    
graph_renderer = from_networkx(G, nx.kamada_kawai_layout, scale=1, center=(0, 0))

color_list=[plt.cm.rainbow(0.7*G.nodes[node]['per_bp_var']) for node in G.nodes]
graph_renderer.node_renderer.data_source.data['colors'] = ["#%02x%02x%02x"%(int(256*color[1]),int(256*color[0]),int(256*color[2])) for color in color_list]


line_widths=np.array([G.edges[edge]['value'] for edge in G.edges])
line_widths=(line_widths-line_widths.min())/(line_widths.max()-line_widths.min())
line_widths=(1+7*line_widths).astype(int)

graph_renderer.edge_renderer.data_source.data['widths'] = line_widths

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='colors')

graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width='widths',line_color="#0000FF")
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_alpha=0.5, line_width='widths',line_color="#1ef20f")
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_alpha=0.5, line_width='widths',line_color="#1ef20f")

#graph_renderer.selection_policy = NodesAndLinkedEdges()
#graph_renderer.inspection_policy = EdgesAndLinkedNodes()

p = figure(plot_width=700, plot_height=700,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1),tools="")
p.renderers.append(graph_renderer)


hover_tool = HoverTool(tooltips=[("Code","@phenotype"),("Phenotype", "@index"), ("Polygenicity", "@per_bp_var_round")])
p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())

url = "/phenotype/@phenotype_lower/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)


p.axis.visible=False
p.legend.visible=False
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text = "Correlation Network"

output_file(web_test_path)
show(p)

In [33]:
bokeh_to_html(p,web_network_path)

with open(web_network_path,'w') as f:
    json.dump(json_item(p),f)

# Polygenicity-h2

In [52]:
h2_total['description']=h2_total.index.map(code_to_description)

In [53]:
var_and_h2=pd.concat([
    per_bp_var['bp8'],
    h2_total['uni.Total Observed scale h2'],
    h2_total['uni.Total Liability scale h2'],
    h2_total['uni.Total h2 p'],
    h2_total['description']
    ],axis=1)

In [91]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import Plot, Range1d, Circle, HoverTool, MultiLine, BoxZoomTool, ResetTool
def bokeh_var_and_h2(var_and_h2,mode='cont'):
    
    p = figure(plot_width=700, plot_height=700,tools="")
    
    if mode=='continuous':
        print('contiuous')
        var_and_h2_filtered=var_and_h2[var_and_h2['uni.Total Liability scale h2']==var_and_h2['uni.Total Observed scale h2']]
        var_and_h2_filtered['x_data']=np.log10(var_and_h2_filtered['uni.Total Observed scale h2'])
        p.xaxis.axis_label = "log(h2) (Observed scale)"
        p.title.text = "Level of Polygenicity - Heritability (continuous traits only)"
    elif mode=='binary':
        print('binary mode')
        var_and_h2_filtered=var_and_h2[var_and_h2['uni.Total Liability scale h2']!=var_and_h2['uni.Total Observed scale h2']]
        var_and_h2_filtered['x_data']=np.log10(var_and_h2_filtered['uni.Total Liability scale h2'])
        p.xaxis.axis_label = "log(h2) (Liability scale)"
        p.title.text = "Level of Polygenicity - Heritability (binary traits only)"
    elif mode=='all':
        print('all mode')
        var_and_h2_filtered=var_and_h2
        var_and_h2_filtered['x_data']=np.log10(var_and_h2_filtered['uni.Total Liability scale h2'])
        p.xaxis.axis_label = "log(h2) (Liability scale)"
        p.title.text = "Level of Polygenicity - Heritability (all traits)"
    else:
        raise

    #var_and_h2_filtered=var_and_h2.copy()


    var_and_h2_filtered['y_data']=var_and_h2_filtered['bp8']
    var_and_h2_filtered['hue_data']=np.log10(var_and_h2_filtered['uni.Total h2 p'])
    var_and_h2_filtered['hue_data']=norm_color(var_and_h2_filtered['hue_data'],plt.cm.viridis)
    var_and_h2_filtered['index_lower']=var_and_h2_filtered.index.str.lower()
    # output to static HTML file

    # add a square renderer with a size, color, and alpha
    #p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="olive", alpha=0.5)
    #p.circle(x='x_data',y='y_data',fill_color='hue_data',radius=0.005,source=var_and_h2_filtered)
    p.circle(x='x_data',y='y_data',fill_color='hue_data',line_color='hue_data',radius=0.008,alpha=0.5,source=var_and_h2_filtered)

    hover_tool = HoverTool(tooltips=[
        ("phenotype","@description"),
        ("log(h2)","@x_data"),
        ("polygenicity","@y_data")
        #("index", "$index"),
        #("(x,y)", "(@x, @y)"),
        #('desc', '@desc'),
    ])

    


    p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
    #p.add_tools(hover_tool,WheelPanTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
    #p.add_tools(TapTool())
    
    url = "/phenotype/@index_lower/"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    #p.axis.visible=False
    #p.legend.visible=False
    #p.xgrid.visible = False
    #p.ygrid.visible = False
    
    p.yaxis.axis_label = "Level of Polygenicity"


    output_file(web_test_path)
    show(p)
    return p


bokeh_to_html(bokeh_var_and_h2(var_and_h2,mode='continuous'),web_poly_h2_cont_path)
bokeh_to_html(bokeh_var_and_h2(var_and_h2,mode='binary'),web_poly_h2_bin_path)
bokeh_to_html(bokeh_var_and_h2(var_and_h2,mode='all'),web_poly_h2_all_path)

"""
with open(web_poly_h2_bin_path,'w') as f:
    json.dump(bokeh_var_and_h2(var_and_h2,'binary'),f)
    
with open(web_poly_h2_cont_path,'w') as f:
    json.dump(bokeh_var_and_h2(var_and_h2,'continuous'),f)
    
with open(web_poly_h2_all_path,'w') as f:
    json.dump(bokeh_var_and_h2(var_and_h2,'all'),f)
"""

contiuous


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

binary mode


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


all mode


"\nwith open(web_poly_h2_bin_path,'w') as f:\n    json.dump(bokeh_var_and_h2(var_and_h2,'binary'),f)\n    \nwith open(web_poly_h2_cont_path,'w') as f:\n    json.dump(bokeh_var_and_h2(var_and_h2,'continuous'),f)\n    \nwith open(web_poly_h2_all_path,'w') as f:\n    json.dump(bokeh_var_and_h2(var_and_h2,'all'),f)\n"

# ANOVA total

In [64]:
def anova_regression_result_times(regression_result,times):
    h2_upscale=pd.DataFrame([(regression_result['h2']/regression_result['h2'].sum())/regression_result['category'].apply(category_to_format,args=('bp_range',)).values,regression_result['category'].apply(category_to_format,args=(times,))]).T    
    h2_upscale_grouped=(h2_upscale.groupby('category')['h2'].apply(list))
    f_statistic,f_statistic_pval=st.f_oneway(*h2_upscale_grouped.values)    
    #print(h2_upscale_grouped)
    return f_statistic,f_statistic_pval

In [65]:
anova_list_list_times=[[phenotype]+list(map(lambda x: anova_regression_result_times(regression_result_list[-1],x),list(range(len(suffix_bp_list_power)-1,0,-1)))) for phenotype,regression_result_list in regression_result_bp_list_dict.items()]

In [66]:
anova_df_f_times=pd.DataFrame([[anova_list[0]]+[anova[0] for anova in anova_list[1:]] for idx,anova_list in enumerate(anova_list_list_times)])
anova_df_f_times=anova_df_f_times.set_index(0)
anova_df_f_times.columns=suffix_bp_list_power[:-1]
anova_df_f_times.head()

anova_df_pval_times=pd.DataFrame([[anova_list[0]]+[anova[1] for anova in anova_list[1:]] for idx,anova_list in enumerate(anova_list_list_times)])
anova_df_pval_times=anova_df_pval_times.set_index(0)
anova_df_pval_times.columns=suffix_bp_list_power[:-1]
anova_df_pval_times.head()

,bp300,bp128,bp64,bp32,bp16
0,,,,,
50_irnt,0.011109,0.003130,0.000012,0.000023,1.009341e-03
20153_irnt,0.013600,0.008790,0.000252,0.000203,8.324001e-04
5097_irnt,0.000013,0.000660,0.037244,0.000137,1.898413e-33
5098_irnt,0.000051,0.001458,0.038098,0.000048,7.970358e-29
30100_irnt,0.098872,0.093447,0.110663,0.292819,1.182984e-04


In [76]:
anova_plot_data=anova_df_pval_times.copy()
anova_plot_data.columns=anova_plot_data.columns.str.replace('bp','')+'Mb'
anova_plot_data.columns=anova_plot_data.columns.str.replace('300Mb','Chr')
anova_plot_data=-np.log10(anova_plot_data.loc[per_bp_var['bp8'].sort_values(ascending=False).index].T)#,cmap=plt.cm.rainbow)
#anova_plot_data.columns=anova_plot_data.columns.map(code_to_description).map(lambda x: description_to_short(x,'abbr_pub_ori'))

anova_plot_data=anova_plot_data.iloc[:,~anova_plot_data.columns.duplicated()]

In [78]:
anova_plot_data=anova_plot_data.iloc[:,:].T

In [69]:
anova_bokeh=heatmap_to_bokehdata(anova_plot_data)

In [70]:
anova_bokeh['color']=norm_color(anova_bokeh['value'],plt.cm.viridis)
anova_bokeh['color']=norm_color(anova_bokeh['value'].where(anova_bokeh['value']<10,10),sns.cm.rocket_r)

In [71]:
anova_bokeh['code_lower']=anova_bokeh['y_data'].str.lower()
anova_bokeh['y_data']=anova_bokeh['y_data'].map(code_to_description)#.map(lambda x: description_to_short(x,'abbr_pub_ori'))

In [72]:
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file

factors_col=anova_bokeh['x_data'].unique()
factors_row=anova_bokeh['y_data'].unique()

x=np.array([[row]*len(factors_col) for row in factors_row]).reshape(-1)
y=np.array([[col]*len(factors_row) for col in factors_col]).reshape(-1)

p = figure(x_range=factors_col, y_range=factors_row,
            x_axis_location='above',y_axis_location='right',
            plot_width=1000,plot_height=10000,
            )

p.rect(x='x_data',y='y_data',color='color',
        source=anova_bokeh,
        width=1,height=1)


hover_tool= HoverTool(tooltips=[
    #("",@)
    #("index", "$index"),
    ("Scale", "@x_data"),
    ("Phenotype", "@y_data"),
    ("-log10(p-value)","@value"),
    #('desc', '@desc'),
])

p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())

url = "/phenotype/@code_lower/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)


p.title.text = "ANOVA result of phenotypes"

output_file(web_test_path)

show(p)
bokeh_to_html(p,web_anova_total_path)
"""
with open(web_anova_total_path,'w') as f:
    json.dump(json_item(p),f)
"""

"\nwith open(web_anova_total_path,'w') as f:\n    json.dump(json_item(p),f)\n"

In [34]:

st.kstest('norm',x)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [47]:
np.random.seed(9543921) # set random seed to get the same result

In [48]:
st.kstest('norm', False, N=100)

KstestResult(statistic=0.09846229615715152, pvalue=0.26942951763759176)

In [84]:
#st.kstest?

In [57]:
rvs1 = st.norm.rvs(size=100, loc=0., scale=1)
rvs2 = st.norm.rvs(size=100, loc=0, scale=1)

In [62]:
for i in range(10):
    rvs1 = st.norm.rvs(size=100, loc=0., scale=1)
    rvs2 = st.norm.rvs(size=1000000, loc=0, scale=1)
    print(st.ks_2samp(rvs1, rvs2))

Ks_2sampResult(statistic=0.0926030000000001, pvalue=0.34229777866562017)
Ks_2sampResult(statistic=0.09740999999999994, pvalue=0.2844056546015654)
Ks_2sampResult(statistic=0.10833300000000001, pvalue=0.17970296664999635)
Ks_2sampResult(statistic=0.05815300000000001, pvalue=0.878354047117466)
Ks_2sampResult(statistic=0.07125799999999999, pvalue=0.6745717645848996)
Ks_2sampResult(statistic=0.06606299999999998, pvalue=0.7616209828130929)
Ks_2sampResult(statistic=0.09726099999999993, pvalue=0.2860889441154117)
Ks_2sampResult(statistic=0.07810300000000003, pvalue=0.5584874957637925)
Ks_2sampResult(statistic=0.06936599999999998, pvalue=0.7067153372483351)
Ks_2sampResult(statistic=0.16129, pvalue=0.009595817644726983)


In [50]:
(0.20833333333333337, 4.6674975515806989e-005)

(0.20833333333333337, 4.667497551580699e-05)

# By region

In [96]:
h2_bp8_df_temp=h2_bp8_df.copy()
edges=h2_bp8_df_temp.T.corr().mask(np.tril(np.ones(h2_bp8_df_temp.T.corr().shape)).astype(np.bool)).stack()
edges=pd.DataFrame([[edge[0][0],edge[0][1],edge[1]] for edge in edges.iteritems()],columns=['source','target','value'])

In [97]:
edges_filtered=edges.loc[(edges['value'] >0.8)]# & (edges['source'] != edges['target'])]

In [98]:
pheno_unique=h2_bp8_df.index.difference(pd.Index(edges_filtered['target'].unique()))

## sort by value per each loci

In [99]:
h2_bp8_df_unique=h2_bp8_df.loc[pheno_unique]

In [100]:
h2_bp8_df_unique.columns=regression_result_bp_list_dict['23115_irnt'][-1]['category'].apply(category_to_format,args=('padding',))

In [101]:
regional_cutoff=h2_bp8_df_unique.applymap(lambda x : x if x>5/(h2_bp8_df_unique.shape[1]) else np.nan)
regional_cutoff_sum=regional_cutoff.sum()

In [102]:
regional_cutoff_bool=h2_bp8_df_unique.applymap(lambda x : True if x>5/(h2_bp8_df_unique.shape[1]) else False)
regional_cutoff_bool_sum=regional_cutoff_bool.sum()

In [103]:
top_loci_index=regional_cutoff_bool_sum.sort_values(ascending=False)[:].index
top_loci_index=sorted(top_loci_index)

## Top

In [104]:
loci_sorted_value=pd.DataFrame([row.sort_values(ascending=False).values for idx,row in regional_cutoff.T.iterrows()],index=regional_cutoff.T.index).T
loci_sorted_value_top=loci_sorted_value[top_loci_index].iloc[:15]

In [105]:
loci_sorted_value_top=loci_sorted_value_top.applymap(lambda x: x*loci_sorted_value_top.shape[1])

In [106]:
loci_values_max,loci_values_min=loci_sorted_value_top.max().max(),loci_sorted_value_top.min().min()

In [107]:
loci_sorted_desc=pd.DataFrame([row.sort_values(ascending=False).index for idx,row in regional_cutoff.T.iterrows()],index=regional_cutoff.T.index).T
loci_sorted_desc_top=loci_sorted_desc[top_loci_index].iloc[:30]

In [189]:
data=heatmap_to_bokehdata(loci_sorted_value_top.T.iloc[::-1,:]).merge(heatmap_to_bokehdata(loci_sorted_desc_top.T.iloc[::-1,:]),on=['x_data','y_data'])
data=data.rename({'value_x': 'value', 'value_y': 'code'}, axis=1)

In [190]:
data['color']=(data['value']-data['value'].mean())/data['value'].std()

data['color']=data['color'].map(lambda x: np.array(plt.cm.viridis(x)))
data['color']=data['color'].map(lambda color: "#%02x%02x%02x"%(int(256*color[1]),int(256*color[0]),int(256*color[2])))

data['description']=data['code'].map(code_to_description)

#data['x_data']=data['x_data'].map(lambda x: category_to_format(x,'chr_bp'))
#data['y_data']='Rank'+(1+data['y_data'].astype(int)).astype(str)
data['y_data']=data['y_data'].map(lambda x: category_to_format(x,'chr_bp'))
data['x_data']='Rank'+(1+data['x_data'].astype(int)).astype(str)

In [191]:
data['description']=data['description'][~data['value'].isnull()]

In [192]:
data['code_lower']=data['code'].str.lower()

In [193]:
data.head()

,x_data,y_data,value,code,color,description,code_lower
0,Rank1,chr22: 48~51Mb,14.425,3466,#e7fe24,Time from waking to first cigarette,3466
1,Rank2,chr22: 48~51Mb,13.014,4283,#e7fe24,Number of rounds of numeric memory test performed,4283
2,Rank3,chr22: 48~51Mb,12.917,5161_irnt,#e7fe24,6mm regularity index (right),5161_irnt
3,Rank4,chr22: 48~51Mb,11.630,20548_7,#c55169,Manifestations of mania or irritability: I was...,20548_7
4,Rank5,chr22: 48~51Mb,9.378,20535,#473f88,Waking too early,20535


In [ ]:
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file

factors_col=data['x_data'].unique()
factors_row=data['y_data'].unique()

x=np.array([[row]*len(factors_col) for row in factors_row]).reshape(-1)
y=np.array([[col]*len(factors_row) for col in factors_col]).reshape(-1)

#tools="pan,wheel_zoom,box_zoom,reset")
p = figure(x_range=factors_col, y_range=factors_row,
            plot_width=500,plot_height=4000,
            toolbar_location="left",tools="",x_axis_location="above")

#hm.add_tools(BoxZoomTool())

p.rect(x='x_data',y='y_data',color='color',
        line_color='#ffffff',
        source=data,
        width=1,height=1)
#hm.rect(x,y,width=1,height=1)


hover_tool = HoverTool(tooltips=[
    #("",@)

    #("index", "$index"),
    ("Locus","@x_data"),
    ("Rank","@y_data"),
    
    ("Code","@code"),
    ("Phenotype","@description"),
    ("relative","@value{1.11}"),
    
    #('desc', '@desc'),
])

p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
url = "/phenotype/@code_lower/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)


p.title.text = "Top 15 Highly Associated Phenotypes for Each Loci."
p.xaxis.major_label_orientation = "vertical"

#hm..start = 0
#@y{1.11}"
output_file(web_test_path)
#toolbar_location=None,
#        tools="hover", tooltips="@category: @value", x_range=(-1, 1.0))
#show(row(hm, dot, sizing_mode="scale_width"))  # open a browser
show(p)

bokeh_to_html(p,web_byregion_path)

In [907]:
"""
with open(web_byregion_path,'w') as f:
    json.dump(json_item(p),f)
"""

# by Phenotype

## Pie plot

In [88]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import Plot, Range1d, Circle, MultiLine
from bokeh.models import TapTool, PanTool, WheelZoomTool,WheelPanTool, HoverTool, BoxZoomTool, ResetTool,LassoSelectTool, UndoTool, RedoTool,SaveTool,ZoomOutTool

In [89]:
def get_color_series_list(regression_result_list):
    #regression_result_list=regression_result_list.copy()[::-1] if reverse_order else regression_result_list.copy()
    regression_result_list=regression_result_list.copy()
    
    color_series_list=[]
    for idx,regression_result in enumerate(regression_result_list):
        print(idx)
        regression_result=regression_result.copy()
        
        regression_result['color']='#ff#ff#ff'
        for chrN in range(1,22+1):
            regression_result_chr=regression_result.loc[regression_result['category'].map(lambda x: category_to_format(x,'chr'))==chrN]
            color_list=[np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.5,np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.8]
            color_list=["#%02x%02x%02x"%(int(256*color[0]),int(256*color[1]),int(256*color[2]))for color in color_list]
            regression_result['color'][regression_result_chr.index]=[color_list[(color_idx)%2] for color_idx in range(len(regression_result_chr.index))]
            
        color_series_list.append(regression_result['color'])
    return color_series_list
        
color_series_list=get_color_series_list(regression_result_bp_list_dict['23115_irnt'])

0


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


1
2
3
4
5


In [198]:
def bokeh_par_pie(regression_result_list,absorb=True,reverse_order=True,color_series_list=None):
    regression_result_list=regression_result_list.copy()[::-1] if reverse_order else regression_result_list.copy()
    
    p = figure(plot_width=900,plot_height=900, title="Landscape of Heritability",
        x_range=(-1.0, 1.0),y_range=(-1.0, 1.0),tools="")
    
    for idx,regression_result in enumerate(regression_result_list):
        regression_result=regression_result.copy()
        
        if color_series_list==None:
            regression_result['color']='#ff#ff#ff'
            for chrN in range(1,22+1):
                regression_result_chr=regression_result.loc[regression_result['category'].map(lambda x: category_to_format(x,'chr'))==chrN]
                color_list=[np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.5,np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.8]
                color_list=["#%02x%02x%02x"%(int(256*color[0]),int(256*color[1]),int(256*color[2]))for color in color_list]
                regression_result['color'][regression_result_chr.index]=[color_list[(color_idx)%2] for color_idx in range(len(regression_result_chr.index))]   
        else:
            regression_result['color']=color_series_list[::-1][idx] if reverse_order else color_series_list[idx]
                
        if absorb:
            regression_result['h2']=absorb_negative(regression_result['h2'])            
        regression_result['h2_ratio']=regression_result['h2']/regression_result['h2'].sum()
        
        regression_result['angle']=regression_result['h2_ratio']*2*np.pi
        
        start_angle=np.insert(regression_result['angle'].cumsum().values[:-1],0,0)
        end_angle=regression_result['angle'].cumsum().values
        regression_result['start_angle']=start_angle
        regression_result['end_angle']=end_angle
        
        regression_result['value1']=regression_result['h2_ratio']*100
        regression_result['value2']=regression_result['h2']
        
        regression_result['category']=regression_result['category'].map(lambda x: category_to_format(x,'chr_bp'))
        
        data=regression_result[['category','color','angle','start_angle','end_angle','value1','value2']]
        """
        #direction, rotation
        data['start_angle']=-data['start_angle']+np.pi/2+2*np.pi
        data['end_angle']=-data['end_angle']+np.pi/2+2*np.pi
        #clipping
        print(data['start_angle'],2*np.pi*(data['start_angle']/(2*np.pi)).astype(int))
        data['start_angle']=data['start_angle']-2*np.pi*(data['start_angle']/(2*np.pi)).astype(int)
        data['end_angle']=data['end_angle']-2*np.pi*(data['end_angle']/(2*np.pi)).astype(int)
        """
        
        #print(idx,data)
        p.annular_wedge(x=0, y=0,
            outer_radius=1-len(regression_result_list)*(0.1)/(len(regression_result_list)-1)*idx,
            inner_radius=1-len(regression_result_list)*(0.1)/(len(regression_result_list)-1)*(idx+1),
            #direction='clock',
            #start_angle='start_angle', end_angle='end_angle',
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white",fill_color='color',legend='category', source=data)
    
    #tools="hover", tooltips="@category <br> @value2 (@value1 %)"
    hover_tool = HoverTool(tooltips="@category <br> @value2 (@value1%)")
    p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
     

    p.axis.visible=False
    p.legend.visible=False
    p.xgrid.visible = False
    p.ygrid.visible = False
    
    output_file(web_test_path)
    show(p)
    #return json_item(p, "par_pie")
    return p
    
#bokeh_par_pie(regression_result_bp_list_dict['23115_irnt'],color_series_list=color_series_list)

In [ ]:
for idx,row in phenotypes_par_filtered.iterrows():
    bokeh_to_html(bokeh_par_pie(regression_result_bp_list_dict[idx],color_series_list=color_series_list),web_phenotype_par_pie_path.format(idx))
    #with open(web_phenotype_par_pie_path.format(idx),'w') as f:
    #    json.dump(bokeh_par_pie(regression_result_bp_list_dict[idx],color_series_list=color_series_list),f)
    print(idx)


## Correlation Rank

In [ ]:
h2_bp8_corr=h2_bp8_df.T.corr()

In [ ]:
"""
var_and_h2_filtered['y_data']=var_and_h2_filtered['bp8']
var_and_h2_filtered['hue_data']=np.log10(var_and_h2_filtered['uni.Total h2 p'])
var_and_h2_filtered['hue_data']=norm_color(var_and_h2_filtered['hue_data'],plt.cm.viridis)
# output to static HTML file

# add a square renderer with a size, color, and alpha
#p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="olive", alpha=0.5)
p.circle(x='x_data',y='y_data',fill_color='hue_data',radius=0.005,source=var_and_h2_filtered)

hover_tool = HoverTool(tooltips=[
    ("phenotype","@description"),
    ("h2","@x_data"),
    ("polygenicity","@y_data")
    #("index", "$index"),
    #("(x,y)", "(@x, @y)"),
    #('desc', '@desc'),
])
"""

In [ ]:
corr_bokeh=heatmap_to_bokehdata(h2_bp8_corr)

In [ ]:
corr_bokeh=corr_bokeh[(corr_bokeh['x_data']=='50_irnt')]
corr_bokeh['color']=norm_color(corr_bokeh['value'],plt.cm.viridis)

In [24]:
corr_bokeh.head()

,x_data,y_data,value,color
0,50_irnt,50_irnt,1.000,#fee724
497,50_irnt,20153_irnt,0.959,#fee724
994,50_irnt,5097_irnt,0.445,#fee724
1491,50_irnt,5098_irnt,0.427,#fee724
1988,50_irnt,30100_irnt,0.260,#24ab82


In [35]:
#corr_bokeh

In [90]:
def bokeh_corr_rank(phenotype):
    corr_bokeh=heatmap_to_bokehdata(h2_bp8_corr)
    corr_bokeh=corr_bokeh[(corr_bokeh['x_data']==phenotype)]
    data=corr_bokeh.sort_values('value',ascending=False).iloc[:50]
    data['code_lower']=data['y_data'].str.lower()
    data['description']=data['y_data'].map(code_to_description)
    data['color']=norm_color(data['value'],plt.cm.viridis,tonorm=False)
    p = figure(y_range=data['description'].iloc[::-1], plot_height=1000,plot_width=1000)

    #p.vbar(x='description',top='value',source=data,width=0.8)
    p.hbar(y='description',right='value',fill_color='color',line_color='color',source=data,height=0.6)


    hover_tool = HoverTool(tooltips=[
    ("Code","@y_data"),    
    ("Phenotype","@description"),
    ("Correlation","@value")
    #("x,y", "@x, @top"),
    #('desc', '@desc'),
    ])
    p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
    url = "/phenotype/@code_lower/"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    
    
    p.title.text = "Correlation with the phenotype"
    #p.xaxis.major_label_orientation = "vertical"
    #p.xaxis.major_label_orientation = "vertical"
    #p.xaxis.major_label_orientation='vertical'
    #p.xgrid.grid_line_color = None
    #p.yaxis.location='right'
    #p.y_range.start = 0
    #p.y_range.flipped = True
    output_file(web_test_path)
    show(p)
    
    return p


bokeh_corr_rank('50_irnt')

Figure(id='83849', ...)

In [ ]:
for idx,row in phenotypes_par_filtered.iterrows():
    bokeh_to_html(bokeh_corr_rank(idx),web_phenotype_corr_rank_path.format(idx))
    #with open(web_phenotype_corr_rank_path.format(idx),'w') as f:
    #    json.dump(bokeh_corr_rank(idx),f)
    print(idx)

In [1075]:

"""
with open(web_correlation_path.format('50_irnt'),'w') as f:
    json.dump(json_item(plot),f)
"""

"\nwith open(web_correlation_path.format('50_irnt'),'w') as f:\n    json.dump(json_item(plot),f)\n"

## ANOVA per each phenotype

In [87]:
from bokeh.io import show, output_file
from bokeh.palettes import Spectral5
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg as df
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from bokeh.embed import components

def bokeh_anova_pheno(anova_df_pval_times,phenotype):

    output_file(web_test_path)

    data=anova_df_pval_times.loc[phenotype]
    #data.columns=['scale','p-value']
    data.index=(data.index.str.replace('bp','')+'Mb').str.replace('300Mb','chr')
    data=-np.log10(data)

    p = figure(x_range=data.index.values, plot_height=500,title="ANOVA result",tools="")
    
    p.vbar(x=data.index.values,top=data.values,width=0.5)

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.axis_label = "Segment Size"
    p.yaxis.axis_label = "-log10(p-value)"
    
    hover_tool = HoverTool(tooltips=[
    #("",@)
    ("Segment Size", "@x"),    
    ("-log10(p-value)", "@top"),
    #('desc', '@desc'),
    ])

    p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
    #p.x_axis_location??

    show(p)
    #return json_item(p, "anova_pheno")
    return p
    
bokeh_anova_pheno(anova_df_pval_times,'23115_irnt')

Figure(id='2210', ...)

In [88]:
web_phenotype_anova_phenotype_path

'web/partitioned_heritability_website/plot_data/phenotype_anova_phenoype_{}'

In [89]:
for idx,row in phenotypes_par_filtered.iterrows():
    bokeh_to_html(bokeh_anova_pheno(anova_df_pval_times,idx),web_phenotype_anova_phenotype_path.format(idx))
    #with open(web_phenotype_anova_phenotype_path.format(idx),'w') as f:
    #    json.dump(bokeh_anova_pheno(anova_df_pval_times,idx),f)
    print(idx)

50_irnt
20153_irnt
5097_irnt
5098_irnt
30100_irnt
5099_irnt
20015_irnt
5132_irnt
4105_irnt
3148_irnt
4100_irnt
30080_irnt
3143_irnt
5085_irnt
23118_irnt
5084_irnt
23098_irnt
30040_irnt
23106_irnt
23104_irnt
1697
23107_irnt
5257_irnt
23109_irnt
23116_irnt
5265_irnt
30110_irnt
30090_irnt
23099_irnt
23119_irnt
23115_irnt
20016_irnt
49_irnt
30290_irnt
30070_irnt
30010_irnt
3062_irnt
2714
30240_irnt
30270_irnt
30260_irnt
30130_irnt
48_irnt
5263_irnt
30210_irnt
20154_irnt
5256_irnt
5255_irnt
30190_irnt
30120_irnt
3063_irnt
2395_1
30000_irnt
12340_irnt
6138_1
5264_irnt
30150
2139_irnt
22147_irnt
2395_4
30140_irnt
30020_irnt
102_irnt
5556
12336_irnt
30180_irnt
4080_irnt
30280_irnt
30030_irnt
1687
5254_irnt
30200_irnt
1717
4079_irnt
5262_irnt
1747_4
22501_irnt
4194_irnt
6150_100
93_irnt
1180
1747_1
2375
3476
20127_irnt
3786_irnt
47_irnt
46_irnt
6177_2
1737
5157_irnt
4282
94_irnt
6153_2
1070
6138_100
20022_irnt
845
3064_irnt
5012
3456
2744
6177_100
2178
1618
20116_0
1080
2887
3436_irnt
6138_2
12

In [110]:
"""
with open(web_anova_total_path,'w') as f:
    json.dump(json_item(p),f)
"""

"\nwith open(web_anova_total_path,'w') as f:\n    json.dump(json_item(p),f)\n"

# Alluvial

In [862]:
def boche_alluvial(phenotype_list,regression_result_list_dict,chr_list):
    regression_result_list_list=[regression_result_list_dict[phenotype].copy() for phenotype in phenotype_list]
    regression_result_chrdict_list_list=[[regression_result_groupchr(regression_result) for regression_result in regression_result_list] for regression_result_list in regression_result_list_list]
    
    scale_idx=3
    
    df_data=pd.concat([pd.concat([regression_result_chrdict_list_list[pheno_idx][scale_idx][chrN]['h2_ratio'] for pheno_idx in range(len(phenotype_list))],axis=1) for chrN in chr_list])
    
    df_index=pd.concat([regression_result_chrdict_list_list[0][scale_idx][chrN]['category'] for chrN in chr_list])
    df_columns=phenotype_list
    df=pd.DataFrame(df_data);df.columns=df_columns;df.index=df_index
    #print(df)
    df=df.apply(absorb_negative)
    #print(df)
    
    
    #df['color']="#ffffff"
    df_color=pd.Series(["#ffffff"]*df.shape[0],index=df.index)
    #print(color_list)
    #print(df)
    df_color=plt.cm.viridis([i for i in range(len(df_color))])
    df_color=["#%02x%02x%02x"%(int(256*color[0]),int(256*color[1]),int(256*color[2])) for color in df_color]
    """
    for chrN in chr_list:
        df_chr=df.loc[df.index.map(lambda x: category_to_format(x,'chr'))==chrN]
        color_list=[np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.5,np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.8]
        color_list=["#%02x%02x%02x"%(int(256*color[0]),int(256*color[1]),int(256*color[2]))for color in color_list]
        df_color[df_chr.index]=[color_list[(color_idx)%2] for color_idx in range(len(df_chr.index))]
    """
    df=df.T
    
    output_file(web_test_path)

    df.index=df.index.map(code_to_description)
    df.columns=df.columns.map(lambda x: category_to_format(x,'chr_bp'))
    names=df.columns.values
    print(df)
    
    p = figure(x_range=df.index.values,plot_height=5000,y_range=(0, 1),tools=[HoverTool(tooltips="@name")])
    p.grid.minor_grid_line_color = '#eeeeee'

    #p.vbar_stack(stackers=names, x='index', color=list(df_color),legend=[value(x) for x in names], source=df,width=1)
    p.varea_stack(stackers=names, x='index', color=df_color,legend=[value(x) for x in names], source=df)

    hover_tool = HoverTool(tooltips=[
    #("",@)

    ("index", "$index"),
    ("Phenotype","@description"),
    ("(x,y)", "(@index, @y_data)"),
    ("ratio","@value"),
    
    #('desc', '@desc'),
    ])

    #p.add_tools(hover_tool,WheelPanTool(), TapTool(),WheelZoomTool(),PanTool(),LassoSelectTool(),BoxZoomTool(),ZoomOutTool(), ResetTool(),SaveTool())
    
    p.legend.visible=False
    show(p)
    
boche_alluvial(['23115_irnt','30100_irnt'],regression_result_bp_list_dict,list(range(1,22+1)))

category                            Chr1 0-32  Chr1 32-64  Chr1 64-96  \
Leg fat percentage (left)            0.011868    0.011868    0.013416   
Mean platelet (thrombocyte) volume   0.017018    0.007844    0.003636   

category                            Chr1 96-128  Chr1 128-160  Chr1 160-192  \
Leg fat percentage (left)              0.009804      0.003612      0.008772   
Mean platelet (thrombocyte) volume     0.012060      0.008972      0.022041   

category                            Chr1 192-224  Chr1 224-249  Chr2 0-32  \
Leg fat percentage (left)               0.011868      0.004644   0.017544   
Mean platelet (thrombocyte) volume      0.029888      0.018274   0.008506   

category                            Chr2 32-64  ...  Chr18 32-64  Chr18 64-78  \
Leg fat percentage (left)             0.020640  ...     0.017544     0.003612   
Mean platelet (thrombocyte) volume    0.009514  ...     0.004224     0.010000   

category                            Chr19 0-32  Chr19 32-59  Chr20

In [843]:
from bokeh.io import show, output_file
from bokeh.plotting import figure



fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=data,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

In [834]:
type(brewer['Spectral'][10])

list

In [156]:
def boche_alluvial(phenotype_list,regression_result_list_dict,chr_list):
    regression_result_list_list=[regression_result_list_dict[phenotype].copy() for phenotype in phenotype_list]
    regression_result_chrdict_list_list=[[regression_result_groupchr(regression_result) for regression_result in regression_result_list] for regression_result_list in regression_result_list_list]
    
    scale_idx=-1
    
    df_data=pd.concat([pd.concat([regression_result_chrdict_list_list[pheno_idx][scale_idx][chrN]['h2_ratio'] for pheno_idx in range(len(phenotype_list))],axis=1) for chrN in chr_list])
    
    df_index=pd.concat([regression_result_chrdict_list_list[0][scale_idx][chrN]['category'] for chrN in chr_list])
    df_columns=phenotype_list
    df=pd.DataFrame(df_data);df.columns=df_columns;df.index=df_index
    #print(df)
    df=df.apply(absorb_negative)
    #print(df)
    #df_color=pd.Series(["#ffffff"]*df.shape[0],index=df.index)
    df['color']="#ffffff"
    #print(color_list)
   
    for chrN in chr_list:
        df_chr=df.loc[df.index.map(lambda x: category_to_format(x,'chr'))==chrN]
        color_list=[np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.5,np.array(plt.cm.get_cmap('hsv')((chrN-1)/22))[:-1]*0.8]
        color_list=["#%02x%02x%02x"%(int(256*color[1]),int(256*color[0]),int(256*color[2]))for color in color_list]
        df['color'][df_chr.index]=[color_list[(color_idx)%2] for color_idx in range(len(df_chr.index))]
    #df=df.reset_index()
    #print(df)
    a=df
    df=df.T*10000
    print(df,len(df))
    output_file(web_test_path)
    #print([x for x in df.columns.values])
    p = figure(x_range=(0,1), y_range=(0, 100))
    p.varea_stack(stackers=df.columns.values,x='index', source=df)
    show(p)
    
    
    N = 10
    df = pd.DataFrame(np.random.randint(10, 100, size=(15, N))).add_prefix('y')
    names = ["y%d" % i for i in range(N)]
    
    df=a.T*1000
    df.index=[i for i in range(len(df.index))]
    #N=len(df.columns)
    names=df.columns.values
    print(df)
    
    p = figure(x_range=(0, len(df)-1), y_range=(0, 800))
    p.grid.minor_grid_line_color = '#eeeeee'

    #print(value(df.columns.values[0]))
    p.varea_stack(stackers=names, x='index', color='#cccccc',legend=[value(x) for x in names], source=df)

    # reverse the legend entries to match the stacked order
    #p.legend[0].items.reverse()
    show(p)
    
boche_alluvial(['23115_irnt','30100_irnt'],regression_result_bp_list_dict,list(range(1,3+1)))

category      bp8.1.0    bp8.1.8   bp8.1.16   bp8.1.24   bp8.1.32   bp8.1.40  \
23115_irnt  40.366199  25.345535  29.951241  21.498096  36.886824  28.877932   
30100_irnt  47.694196  53.495174   4.462164  60.115499   5.472829  52.745471   

category     bp8.1.48   bp8.1.56   bp8.1.64   bp8.1.72  ...  bp8.3.120  \
23115_irnt  15.242532  36.998531  34.190940  48.080124  ...   30.90821   
30100_irnt  17.537306   6.633025   7.508907   3.151127  ...  181.33381   

category    bp8.3.128  bp8.3.136  bp8.3.144  bp8.3.152  bp8.3.160  bp8.3.168  \
23115_irnt  37.286291  27.794327  11.971642  32.403121  12.264550  44.534344   
30100_irnt  11.250046  22.450521  14.057539   1.827718  28.800871  20.300144   

category    bp8.3.176  bp8.3.184  bp8.3.192  
23115_irnt  21.432719  24.087422  17.916796  
30100_irnt  28.454697   2.095153  31.764859  

[2 rows x 86 columns] 2
category  bp8.1.0   bp8.1.8  bp8.1.16  bp8.1.24  bp8.1.32  bp8.1.40  bp8.1.48  \
0         4.03662  2.534554  2.995124   2.14981  3.

## Table Data

In [233]:
data_merged=phenotypes_par_filtered.merge(h2_total[h2_total.columns[h2_total.columns.str.contains('uni')]],on='phenotype')
data_merged.columns=data_merged.columns.str.replace('uni.','')

In [234]:
data_merged['description'].loc['C_SKIN']='C_SKIN'

In [232]:
'aa'.upper()

'AA'

In [235]:
data_merged['description']=data_merged.apply(lambda x: '<a href="/phenotype/{}">'.format(x.name.lower())+x['description']+'</a>',axis=1)
data_merged['-log10(h2_p)']=-np.log10(data_merged['Total h2 p']).round(2)
data_merged['prev(%)']=(data_merged['n_cases']/(data_merged['n_cases']+data_merged['n_controls'])*100).round(2)
data_merged['Mean Chi^2']=data_merged['Mean Chi^2'].round(2)
data_merged['Lambda GC']=data_merged['Lambda GC'].round(2)
data_merged['Intercept']=data_merged['Intercept'].round(2)

In [236]:
data_merged.shape

(497, 29)

In [237]:
data_merged=data_merged[[ 'description','Sex',
                'Total Liability scale h2','-log10(h2_p)',
               'Intercept', 'Intercept_se', 'Ratio','Lambda GC','Mean Chi^2', 
               'prev(%)'
              ]]

In [238]:
tosave=data_merged.rename(mapper={'description':'Description','Total Liability scale h2':'h2'},axis='columns')

In [239]:
tosave.to_csv(web_data_path+'table.csv')

# Phenotype Info

In [191]:
data_merged=phenotypes_par_filtered.merge(h2_total[h2_total.columns[h2_total.columns.str.contains('uni')]],on='phenotype')
data_merged.columns=data_merged.columns.str.replace('uni.','')

In [192]:
#data_merged['description']=data_merged.apply(lambda x: '<a href="/phenotype/{}">'.format(x.name)+x['description']+'</a>',axis=1)
data_merged['UK Biobank Data Showcase Link']=data_merged['UK Biobank Data Showcase Link'].map(lambda x: '<a href="{}">{}</a>'.format(x,x) if type(x)==str else '')
#=data_merged.apply(lambda x: '<a href="{}">'.format(x['UK Biobank Data Showcase Link'])+x['UK Biobank Data Showcase Link']+'</a>' if type(x),axis=1)

data_merged['-log10(h2_p)']=-np.log10(data_merged['Total h2 p']).round(2)
data_merged['prev(%)']=(data_merged['n_cases']/(data_merged['n_cases']+data_merged['n_controls'])*100).round(2)
data_merged['Mean Chi^2']=data_merged['Mean Chi^2'].round(2)
data_merged['Lambda GC']=data_merged['Lambda GC'].round(2)
data_merged['Intercept']=data_merged['Intercept'].round(2)

In [193]:
#data_merged['UK Biobank Data Showcase Link'].iloc[1]

In [194]:
data_merged.columns

Index(['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',
       'wget command', 'Dropbox File', 'filename', 'Total Observed scale h2',
       'Total Observed scale h2_se', 'Total Liability scale h2',
       'Total Liability scale h2_se', 'Lambda GC', 'Mean Chi^2', 'Intercept',
       'Intercept_se', 'Ratio', 'Total h2 z', 'Total h2 p', '-log10(h2_p)',
       'prev(%)'],
      dtype='object')

In [195]:
data_merged=data_merged[['description', 'notes', 'Sex', 
       'Total Observed scale h2', 'Total Observed scale h2_se','Total Liability scale h2', 'Total Liability scale h2_se','-log10(h2_p)',
        'Intercept', 'Intercept_se','Ratio','Lambda GC','Mean Chi^2',  
        'prev(%)','n_missing', 'n_non_missing', 'n_cases', 'n_controls',               
       'UK Biobank Data Showcase Link', 'variable_type'
        ]]

In [196]:
tosave=data_merged

In [197]:
tosave.to_csv(web_data_path+'table_all.csv')

# Template Page generation

In [220]:
def replace_file(path_in,path_out,mapping_list):
    f=open(path_in,'r')
    f_read=f.read()
    f.close()
    
    for source,target in mapping_list:
        f_read=f_read.replace(source,target)
    
    f=open(path_out,'w')
    f.write(f_read)
    f.close()
    print("file replaced",path_in,path_out)
        

In [208]:
phenotypes_par_filtered.index[phenotypes_par_filtered.index.str.contains('M13')]

Index(['M13_ARTHROSIS', 'M13_OTHERJOINT', 'M13_MENISCUSDERANGEMENTS'], dtype='object', name='phenotype')

In [209]:
phenotypes_par_filtered.loc['M13_OTHERJOINT']['description']

'#Other joint disorders'

In [211]:
web_path

'web/partitioned_heritability_website/'

In [229]:
phenotypes_par_filtered.loc['C_SKIN','description']='C_SKIN'

phenotypes_par_filtered.loc['C_SKIN']

PHESANT_transformation                                                         NaN
Sex                                                                     both_sexes
description                                                                 C_SKIN
n_cases                                                                      16531
n_controls                                                                  344663
n_missing                                                                        0
n_non_missing                                                               361194
notes                                                                          NaN
source                                                                     finngen
variable_type                                                               binary
Phenotype Code                                                              C_SKIN
UK Biobank Data Showcase Link                                                  NaN
File

In [ ]:
cnt=0
for idx,row in phenotypes_par_filtered.iterrows():
    target_1=idx
    target_2=row['description'].replace(':','.')
    target_2=target_2.replace('#','')
    cnt+=1
    print(cnt)
    replace_file(web_path+'content/phenotype/template.rmd',web_path+'content/phenotype/{}.rmd'.format(idx),
                 
             [('23115_irnt',target_1),('Leg fat percentage',target_2)]
            )
    #bokeh_to_html(bokeh_anova_pheno(anova_df_pval_times,idx),web_phenotype_anova_phenotype_path.format(idx))
    #with open(web_phenotype_anova_phenotype_path.format(idx),'w') as f:
    #    json.dump(bokeh_anova_pheno(anova_df_pval_times,idx),f)